In [1]:
import os
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd ./drive/MyDrive/CommonSwissBirds

/content/drive/MyDrive/CommonSwissBirds


In [4]:
%ls

01_data_prep/   04_resnet_50/         07_VGG16/       utils/
02_data/        05_efficientNetV2L/   08_custom_CNN/
03_base_model/  06_mobileNetV3_mini/  readme.ipynb


In [5]:
sys.path.append('./utils')
import utils

In [6]:
print(f"The number of GPUs: {len(tf.config.list_physical_devices('GPU'))}")
!nvidia-smi -L

The number of GPUs: 1
GPU 0: Tesla T4 (UUID: GPU-cc3812f2-0a12-9a31-c3b2-e81b239a153e)


In [7]:
class_names=['amsel', 'blaumeise', 'buchfink', 'elster', 'hausrotschwanz', 'hausspatz', 'kohlmeise', 'rabenkraehe', 'star', 'tuerkentaube']

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './02_data/99_dataset_preparation/train_images',
    labels='inferred',
    label_mode='categorical',
    class_names=class_names,
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="training",
)

Found 5000 files belonging to 10 classes.
Using 4000 files for training.


In [9]:
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './02_data/99_dataset_preparation/train_images',
    labels='inferred',
    label_mode='categorical',
    class_names=class_names,
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation",
)

Found 5000 files belonging to 10 classes.
Using 1000 files for validation.


In [10]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './02_data/99_dataset_preparation/test_images',
    labels='inferred',
    label_mode='categorical',
    class_names=class_names,
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=False,
)

Found 1222 files belonging to 10 classes.


In [11]:
showcase_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './02_data/99_dataset_preparation/showcase_images',
    labels='inferred',
    label_mode='categorical',
    class_names=class_names,
    color_mode='rgb',
    batch_size=32,
    shuffle=False,
    image_size=(224, 224),
)

Found 30 files belonging to 10 classes.


In [12]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)
showcase_ds = showcase_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [14]:
num_classes = 10
#Build exactly VGG16....
def createModel():
    cam_model = tf.keras.Sequential(name="caminada_nn")
   
    cam_model.add(tf.keras.layers.Conv2D(64, 3, name='block1_conv1', activation="relu", padding="same"))
    cam_model.add(tf.keras.layers.Conv2D(64, 3, name='block1_conv2', activation="relu", padding="same"))
    cam_model.add(tf.keras.layers.MaxPooling2D(strides=(2,2), name='block1_pool'))

    cam_model.add(tf.keras.layers.Conv2D(128, 3, name='block2_conv1', activation="relu", padding="same"))
    cam_model.add(tf.keras.layers.Conv2D(128, 3, name='block2_conv2', activation="relu", padding="same"))
    cam_model.add(tf.keras.layers.MaxPooling2D(strides=(2,2), name='block2_pool'))

    cam_model.add(tf.keras.layers.Conv2D(256, 3, name='block3_conv1', activation="relu", padding="same"))
    cam_model.add(tf.keras.layers.Conv2D(256, 3, name='block3_conv2', activation="relu", padding="same"))
    cam_model.add(tf.keras.layers.Conv2D(256, 3, name='block3_conv3', activation="relu", padding="same"))
    cam_model.add(tf.keras.layers.MaxPooling2D(strides=(2,2), name='block3_pool'))

    cam_model.add(tf.keras.layers.Conv2D(512, 3, name='block4_conv1', activation="relu", padding="same"))
    cam_model.add(tf.keras.layers.Conv2D(512, 3, name='block4_conv2', activation="relu", padding="same"))
    cam_model.add(tf.keras.layers.Conv2D(512, 3, name='block4_conv3', activation="relu", padding="same"))
    cam_model.add(tf.keras.layers.MaxPooling2D(strides=(2,2), name='block4_pool'))

    cam_model.add(tf.keras.layers.Conv2D(512, 3, name='block5_conv1', activation="relu", padding="same"))
    cam_model.add(tf.keras.layers.Conv2D(512, 3, name='block5_conv2', activation="relu", padding="same"))
    cam_model.add(tf.keras.layers.Conv2D(512, 3, name='block5_conv3', activation="relu", padding="same"))
    cam_model.add(tf.keras.layers.MaxPooling2D(strides=(2,2), name='block5_pool'))

    cam_model.add(tf.keras.layers.GlobalAveragePooling2D(name='global_average_pooling2d'))
    cam_model.add(tf.keras.layers.Dense(1024, name='dense_2', activation="relu"))
    cam_model.add(tf.keras.layers.Dense(num_classes, name='dense_3', activation="softmax"))

    cam_model.compile(
        optimizer='adam',
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
        metrics=['accuracy'])
    return cam_model
model = createModel()

In [18]:
model.summary()

Model: "caminada_nn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       

In [16]:
callbacks = [EarlyStopping(monitor='val_accuracy', mode = "max", patience=3),
ModelCheckpoint(
    filepath=r'./08_custom_CNN/model/best.h5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)]

In [17]:
history = model.fit(
  train_ds,
  validation_data=valid_ds,
  epochs=50,
  callbacks=[callbacks],
)

Epoch 1/50
125/125 [==============================] - 445s 3s/step - loss: 3.4823 - accuracy: 0.0995 - val_loss: 2.3018 - val_accuracy: 0.0910
Epoch 2/50
125/125 [==============================] - 53s 424ms/step - loss: 2.3028 - accuracy: 0.1053 - val_loss: 2.3018 - val_accuracy: 0.0910
Epoch 3/50
125/125 [==============================] - 54s 432ms/step - loss: 2.3027 - accuracy: 0.1053 - val_loss: 2.3017 - val_accuracy: 0.0910
Epoch 4/50
125/125 [==============================] - 54s 436ms/step - loss: 2.3026 - accuracy: 0.1053 - val_loss: 2.3018 - val_accuracy: 0.0910


In [ ]:
%matplotlib inline
utils.plot_history(history)

In [ ]:
# Possible to start here without training... just loading the new weights...
model = createModel()
model.build(input_shape=(None,224,224,3))
model.load_weights('./08_custom_CNN/model/best.h5')

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_ds, batch_size=128)
print("test loss, test acc:", results)

In [ ]:
testset_predictions = model.predict(test_ds)
print(testset_predictions)
print(testset_predictions.shape)

In [ ]:
testset_predictions = model.predict(test_ds)
predicted_labels = [prediction.argmax() for prediction in testset_predictions]
true_labels = utils.get_labels_from_int_batch_dataset(test_ds)
confusion_matrix = np.array(tf.math.confusion_matrix(true_labels, predicted_labels))

In [ ]:
print(classification_report(true_labels, predicted_labels, target_names=class_names))

In [ ]:
%matplotlib inline
utils.plot_sns_confusion_matrix(confusion_matrix, class_names)

In [ ]:
showcase_predictions = model.predict(showcase_ds)
predicted_labels = [prediction.argmax() for prediction in showcase_predictions]
true_labels = [label.numpy() for label in [labels for images, labels in showcase_ds.take(1)][0]]
confusion_matrix = np.array(tf.math.confusion_matrix(true_labels, predicted_labels))

In [ ]:
print(classification_report(true_labels, predicted_labels, target_names=class_names))

In [ ]:
%matplotlib inline
utils.plot_sns_confusion_matrix(confusion_matrix, class_names)

In [ ]:
%matplotlib inline
utils.plot_showcase(showcase_ds, class_names, showcase_predictions)

In [ ]:
model.summary()